# Trabajo Especial - Modelos y Simulación 2025
### Lucas Nieto y Alvaro Santiago Medina
---

# Problema

Se desea estimar mediante simulación estocástica la siguiente integral definida sobre el hipercubo $[0,1]^d$, donde $d=2,5,10$:
$$
I_d = \int_{[0,1]^d} \prod_{i=1}^d e^{-x_i^2}dx
$$

Como la integral es separable y tiene un valor exacto conocido, viene dado por:
$$
I_d = \bigg( \int_0^1 e^{-x_i^2}dx \bigg)^d = \bigg(\frac{\sqrt{\pi}}{2} \cdot erf(1)\bigg)
$$

Donde *erf* es la función error valuada en 1. Tal que:
$$
erf(1) = \frac{2}{\sqrt{\pi}}\int_0^1 e^{-t^2}dt \approx 0.8427
$$

Luego el valor de $I_d$ es tal que:
$$
I_d = \bigg(\frac{\sqrt{\pi}}{2} \cdot erf(1)\bigg) = \bigg(\frac{\sqrt{\pi}}{2} \cdot 0.8427 \bigg) = \boxed{0.74682}
$$

Para realizar la estimación se utilizará *Monte Carlo* con un número fijo de estimaciones.

Como Monte Carlo requiere de un generador de muestras uniformes en el $(0,1)$ se utilizarán los siguientes generadores:
- **Congruencial Lineal Mixto con parámetro**s $(a, c, m) = (16807, 0, 2^{31}-1)$
- **XORshift** 
- **Mersenne Twister** (MT19937)

Dichas implementaciones para los generadores podemos encontrarlas en la carpeta 📁`rngs/`

# Importaciones Generales

In [1]:
from constants import *
import numpy as np
import matplotlib.pyplot as plt
from MonteCarlo import MonteCarlo
from Utils import Utils
from rngs.Xorshift32 import Xorshift
from rngs.MersenneTwister import MersenneTwister
from rngs.LCG import LCG
from time import time

%matplotlib inline

# Generadores

Para asegurarnos de que la implementación de cada uno de ellos fué correcta realizamos gráficos para chequear que ninguno cae en el problema de los *hiperplanos*.

- Graficar para cada generador en 2D con función en utils
- Graficar hipercubo para ver que no hay hiperplanos

# Estimaciones

Para que los resultados sean comparables seteamos una semilla general

In [2]:
SEED = int(time())

## Generador Congruencial Lineal

Generamos una instancia de para la clase de *Generador Congruencial Lineal* con la `seed` general

In [3]:
lcg = LCG(seed=SEED)

Seguido de esto realizamos la estimación de la integral utilizando Monte Carlo.

In [4]:
lcg_estimation = Utils.rng_estimation_gaussian_in_hipercube(
                    Nsamples=SAMPLE_SIZE_SMALL,
                    rng=lcg,
                    d=2
                )

lcg_cuad_error = Utils.rng_cuadratic_error_estimation(
                    Nsim=1, 
                    Nsamples=SAMPLE_SIZE_SMALL, 
                    rng=lcg,
                    d=2
                )

#Valor conocido de la integral
print(f"🤔 ESTIMACIÓN LCG -> {lcg_estimation:.6f}")
print(f"🎯 VALOR REAL -> {INTEGRAL_VAL_D2:.6f}")
print(f"🔴 ERROR CUADRÁTICO MEDIO de LCG -> {lcg_cuad_error:.6f}")

🤔 ESTIMACIÓN LCG -> 0.557978
🎯 VALOR REAL -> 0.557740
🔴 ERROR CUADRÁTICO MEDIO de LCG -> 0.000001


## Xorshift

Generamos una instancia de para la clase de *Xorshift* con la `seed` general

In [5]:
xorshift  = Xorshift(seed=SEED)

In [6]:
xorshift_estimation = Utils.rng_estimation_gaussian_in_hipercube(
                        Nsamples=SAMPLE_SIZE_SMALL,
                        rng=xorshift,
                        d=2
                    ) 

xorshift_cuad_error = Utils.rng_cuadratic_error_estimation(
                        Nsim=1, 
                        Nsamples=SAMPLE_SIZE_SMALL, 
                        rng=xorshift,
                        d=2
                    )

print(f"🤔 ESTIMACIÓN XORSHIFT -> {xorshift_estimation:.6f}")
print(f"🎯 VALOR REAL -> {INTEGRAL_VAL_D2:.6f}")
print(f"🔴 ERROR CUADRÁTICO MEDIO de XORSHIFT -> {xorshift_cuad_error:.6f}")

🤔 ESTIMACIÓN XORSHIFT -> 0.557111
🎯 VALOR REAL -> 0.557740
🔴 ERROR CUADRÁTICO MEDIO de XORSHIFT -> 0.000003


## Mersenne Twister

Generamos una instancia de para la clase de *Mersenne Twister* con la `seed` general

In [7]:
mersenne_twister = MersenneTwister(seed_value=SEED)

Realizamos la estimación de la integral utilizando Monte Carlo.

In [8]:
mersenne_twister_estimation = Utils.rng_estimation_gaussian_in_hipercube(
                                Nsamples=SAMPLE_SIZE_SMALL,
                                rng=mersenne_twister,
                                d=2
                            )

mersenne_twister_cuad_error = Utils.rng_cuadratic_error_estimation(
                                Nsim=1, 
                                Nsamples=SAMPLE_SIZE_SMALL, 
                                rng=mersenne_twister,
                                d=2
                            )

#Resultado Teórico
print(f"🤔 ESTIMACIÓN MERSENNE TWISTER -> {mersenne_twister_estimation:.6f}")
print(f"🎯 VALOR REAL -> {INTEGRAL_VAL_D2:.6f}")
print(f"🔴 ERROR CUADRÁTICO MEDIO de MT -> {mersenne_twister_cuad_error:.6f}")

🤔 ESTIMACIÓN MERSENNE TWISTER -> 0.554772
🎯 VALOR REAL -> 0.557740
🔴 ERROR CUADRÁTICO MEDIO de MT -> 0.000007


# Análisis

## Tiempo de estimación

In [9]:
lcg_estimation = MonteCarlo.time_method(Nsamples=SAMPLE_SIZE_SMALL,
                                        g=Utils.gaussian_function,
                                        rng=lcg)
print(f"tiempo de estimación: {lcg_estimation[0]:.6f} secs")
print(f"estimación: {lcg_estimation[1]:.6f}")

tiempo de estimación: 0.010354 secs
estimación: 0.746130


In [10]:
sample_results = Utils.compare_muestral_stats(Nsamples=SAMPLE_SIZE_SMALL, Nsim=100, seed=int(time()), d=2)
print(f"{sample_results["LCG"][1]:.6f}")
print(f"{sample_results["Xorshift"][1]:.6f}")
print(f"{sample_results['Xorshift'][1]:.6f}")

0.000009
0.000010
0.000010
